In [1]:
from IPython import get_ipython
from typing_extensions import Annotated
from dotenv import load_dotenv, find_dotenv

import autogen
from autogen.cache import Cache
import os

load_dotenv(find_dotenv())

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4o", "gpt4-turbo"],
    },
)

settings = {
    'host': os.environ.get('COSMOS_ENDPOINT'),
    'master_key': os.environ.get('ACCOUNT_KEY'),
    'database_id': os.environ.get('COSMOS_DATABASE'),
    'container_id': os.environ.get('COSMOS_CONTAINER'),
}

In [9]:
llm_config = {"config_list": config_list, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="TERMINATE",
    code_execution_config=False
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="Create a plan to troubleshoot the issue and solve the problem of the user.",
    llm_config=llm_config,
)
troubleshooter = autogen.AssistantAgent(
    name="Troubleshooter",
    system_message="Identify the issue with the user's system based on the available data and provide a solution.",
    llm_config=llm_config,
)
data_engineer = autogen.AssistantAgent(
    name="Data Engineer",
    system_message="Write the code to retrieve the needed data, summarize the data, and provide it to the troubleshooter.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, planner, troubleshooter, data_engineer], messages=[], max_round=15)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [13]:
import tempfile

from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a Docker command line code executor.
executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # Execute code using the given docker image name.
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration that uses docker.
code_executor_agent_using_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the docker command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

# When the code executor is no longer used, stop it to release the resources.
# executor.stop()

In [10]:
@user_proxy.register_for_execution()
@executor.register_for_llm(name="python", description="run cell in ipython and return the execution result.")
def exec_python(cell: Annotated[str, "Valid Python cell to execute."]) -> str:
    ipython = get_ipython()
    result = ipython.run_cell(cell)
    log = str(result.result)
    if result.error_before_exec is not None:
        log += f"\n{result.error_before_exec}"
    if result.error_in_exec is not None:
        log += f"\n{result.error_in_exec}"
    return log

In [ ]:
import json
import requests
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import json

HOST = settings['host']
MASTER_KEY = settings['master_key']
DATABASE_ID = settings['database_id']
CONTAINER_ID = settings['container_id']

# Initialize the Cosmos client
client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY})

# Get the database and container
db = client.get_database_client(DATABASE_ID)
container = db.get_container_client(CONTAINER_ID)

def get_customer_info(customer_id):
    try:
        # Query the container for the customer with the specified ID
        query = f"SELECT * FROM c WHERE c.id = '{customer_id}'"
        items = list(container.query_items(query=query, enable_cross_partition_query=True))
        
        if items:
            # If the customer is found, return the customer information as JSON
            customer_info = items[0]
            return json.dumps(customer_info, default=str)
        else:
            # If the customer is not found, return an error message as JSON
            error_message = {'error': f"Customer with ID '{customer_id}' not found."}
            return json.dumps(error_message)
    
    except exceptions.CosmosHttpResponseError as e:
        # If there's an error querying the container, return the error message as JSON
        error_message = {'error': str(e)}
        return json.dumps(error_message)


def get_outage_info(customer_address):
    # Extract the city from the customer's address
    city = customer_address.split(', ')[1]
    
    # Define the outage information for each city
    outage_info = {
        'New York': {
            'status': 'outage',
            'message': 'There is currently a mass outage in New York affecting multiple services.',
            'affected_services': ['Internet', 'Phone', 'TV'],
            'estimated_resolution_time': '2 days'
        },
        'Chicago': {
            'status': 'healthy',
            'message': 'All services are operating normally in Chicago.',
            'affected_services': [],
            'estimated_resolution_time': None
        },
        'San Francisco': {
            'status': 'healthy',
            'message': 'All services are operating normally in San Francisco.',
            'affected_services': [],
            'estimated_resolution_time': None
        },
        'Munich': {
            'status': 'outage',
            'message': 'There is currently a mass outage in Munich affecting Internet and Phone services.',
            'affected_services': ['Internet', 'Phone'],
            'estimated_resolution_time': '1 day'
        },
        'Berlin': {
            'status': 'healthy',
            'message': 'All services are operating normally in Berlin.',
            'affected_services': [],
            'estimated_resolution_time': None
        }
    }
    
    # Get the outage information for the customer's city
    city_outage_info = outage_info.get(city)
    
    if city_outage_info:
        # If the city is found, return the outage information as JSON
        return json.dumps(city_outage_info)
    else:
        # If the city is not found, return a default message as JSON
        default_message = {
            'status': 'unknown',
            'message': f"No outage information available for {city}.",
            'affected_services': [],
            'estimated_resolution_time': None
        }
        return json.dumps(default_message)


In [ ]:
autogen.agentchat.register_function(
    exec_python,
    caller=chatbot,
    executor=user_proxy,
    name="sh",
    description="run a shell script and return the execution result.",
)

In [ ]:
user_proxy.initiate_chat(
    manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
)